In [1]:
import math
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
from scipy.special import softmax

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("always", ConvergenceWarning)

In [2]:
from maml.datasets.miniimagenet import MiniimagenetMetaDataset
from maml.models.gated_conv_net import ImpRegConvModel
from maml.models.conv_embedding_model import RegConvEmbeddingModel
from maml.logistic_regression_utils import logistic_regression_grad_with_respect_to_w, logistic_regression_hessian_pieces_with_respect_to_w, logistic_regression_hessian_with_respect_to_w, logistic_regression_mixed_derivatives_with_respect_to_w_then_to_X

In [3]:
modulation_mat_rank = 32
num_channels = 32
dataset = MiniimagenetMetaDataset(
    root='data',
    img_side_len=84,
    num_classes_per_batch=5,
    num_samples_per_class=5,
    num_total_batches=5,
    num_val_samples=5,
    meta_batch_size=2,
    split='train',
    num_workers=4,
    device='cuda')
model = ImpRegConvModel(
        input_channels=dataset.input_size[0],
        output_size=dataset.output_size,
        num_channels=num_channels,
        modulation_mat_rank=modulation_mat_rank,
        img_side_len=dataset.input_size[1],
        use_max_pool=False,
        verbose=False)

MiniImagenet train


In [4]:
modulation_mat_size = (modulation_mat_rank, num_channels*8)
embedding_hidden_size = 256
embedding_num_layers = 2
num_conv_embedding_layer = 4

embedding_model = RegConvEmbeddingModel(
             input_size=np.prod(dataset.input_size),
             output_size=dataset.output_size,
             modulation_mat_size=modulation_mat_size,
             hidden_size=embedding_hidden_size,
             num_layers=embedding_num_layers,
             convolutional=True,
             num_conv=num_conv_embedding_layer,
             num_channels=num_channels,
             rnn_aggregation=False,
             linear_before_rnn=False,
             embedding_pooling='max',
             batch_norm=True,
             avgpool_after_conv=True,
             num_sample_embedding=0,
             sample_embedding_file='abcdf',
             img_size=dataset.input_size,
             verbose=False,
             original_conv=False,
             modulation_mat_spec_norm = 10000)

In [5]:
slow_lr = 0.001
optimizer_specs = \
        [{'params': model.parameters(), 'lr': slow_lr},
         {'params': embedding_model.parameters(), 'lr': slow_lr}]

In [6]:
l2_lambda = 2

In [7]:
model.to('cuda')

ImpRegConvModel(
  (features): Sequential(
    (layer1_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (layer1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
    (layer1_relu): ReLU(inplace=True)
    (layer2_conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (layer2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
    (layer2_relu): ReLU(inplace=True)
    (layer3_conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (layer3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
    (layer3_relu): ReLU(inplace=True)
    (layer4_conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (layer4_bn): BatchNorm2d(256, eps=1e-05, momentum=0.001, affine=False, track_running_stats=True)
    (layer4_relu): ReLU(inplace=True)
  )
)

In [8]:
l2_lambda

2

In [9]:
embedding_model.to('cuda')

In [10]:
for train_task_batch, test_task_batch in iter(dataset):
    break

In [11]:
train_task = train_task_batch[0]

In [12]:
modulation = embedding_model(train_task, return_task_embedding=False)

In [13]:
# old_params = model.classifier['fully_connected'].weight.cuda()
features = model(
    train_task.x, modulation=modulation)

In [14]:
features.shape

torch.Size([25, 33])

In [15]:
X_train = features.detach().cpu().numpy()
y_train = (train_task.y).cpu().numpy()

In [16]:
with warnings.catch_warnings(record=True) as w:
    lr_model = LogisticRegression(solver='lbfgs', penalty='l2', 
        C=1/(l2_lambda), #????????????????????????????????????????
        tol=1e-6, max_iter=100,
        multi_class='multinomial', fit_intercept=False)
    lr_model.fit(X_train, y_train)

In [17]:
lr_model.predict(X_train)

array([0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3])

In [18]:
y_train

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3])

In [19]:
w = lr_model.coef_.astype('float32')

In [20]:
w.dtype

dtype('float32')

In [21]:
inner_loss_func = torch.nn.CrossEntropyLoss()

In [22]:
grad_train_np = logistic_regression_grad_with_respect_to_w(X_train, y_train, w)

In [23]:
grad_train_np.dtype

dtype('float64')

In [24]:
hessian_np = logistic_regression_hessian_with_respect_to_w(X_train, y_train, w)

In [25]:
def compute_hessian(X, y, w):
    lr_hessian = logistic_regression_hessian_with_respect_to_w(X, y, w)
    hessian = lr_hessian + l2_lambda * np.eye(lr_hessian.shape[0])
    return hessian

In [36]:
def compute_inverse_hessian(X, y, w):
    diag, Xbar = logistic_regression_hessian_pieces_with_respect_to_w(X, y, w)
    pre_inv = np.matmul(Xbar, Xbar.T) + l2_lambda * np.diag(np.reciprocal(diag))
    inv = np.linalg.inv(pre_inv)
    return 1 / l2_lambda * (np.eye(Xbar.shape[1]) - np.matmul(np.matmul(Xbar.T, inv), Xbar))

In [37]:
hessian_plus_np = compute_hessian(X_train, y_train, w)

In [38]:
hessian_inv_np = compute_inverse_hessian(X_train, y_train, w)

In [62]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
for i in range(1000):
    hessian_plus_np = compute_hessian(X_train, y_train, w)
elapsed = timeit.default_timer() - start_time

In [63]:
elapsed

2.7932463879697025

In [27]:
J = logistic_regression_mixed_derivatives_with_respect_to_w_then_to_X(X_train, y_train, w)

In [72]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
for i in range(1000):
    J = logistic_regression_mixed_derivatives_with_respect_to_w_then_to_X(X_train, y_train, w)
elapsed = timeit.default_timer() - start_time

In [73]:
elapsed

2.5134298340417445

In [40]:
adapted_params = torch.tensor(w, requires_grad=True, device='cuda', dtype=torch.float32)

In [41]:
adapted_params.shape

torch.Size([5, 33])

In [42]:
features = model(train_task.x, modulation=modulation)
preds = F.linear(features, weight=adapted_params)
loss = inner_loss_func(preds, train_task.y)
l2_loss = 0.5 * l2_lambda * adapted_params.pow(2).sum()
loss += l2_loss
grad = torch.autograd.grad(loss, adapted_params,
            create_graph=True, allow_unused=False)[0]
grad = grad.flatten()
hessian = torch.zeros((len(grad), len(grad)), device=grad.device)
for i, g in enumerate(grad):
    hessian[i, :] = torch.autograd.grad(g, adapted_params,
            create_graph=False, allow_unused=False, retain_graph=True)[0].flatten()

In [47]:
hessian_inv_torch = np.linalg.inv(hessian.cpu().numpy())

In [46]:
hessian_inv_np

array([[ 4.95125784e-01, -7.52294066e-03,  3.63468205e-03, ...,
         2.89681807e-04, -3.36625560e-04,  4.32487057e-04],
       [-7.52294066e-03,  4.83586179e-01,  1.43760154e-02, ...,
         6.03345372e-04, -2.68417652e-03,  1.64819377e-03],
       [ 3.63468205e-03,  1.43760154e-02,  4.62696923e-01, ...,
         8.15952512e-05,  7.22851534e-03, -3.52438023e-03],
       ...,
       [ 2.89681807e-04,  6.03345372e-04,  8.15952512e-05, ...,
         4.98344866e-01,  5.48840585e-04, -3.72843121e-04],
       [-3.36625560e-04, -2.68417652e-03,  7.22851534e-03, ...,
         5.48840585e-04,  4.77964077e-01,  9.66437798e-03],
       [ 4.32487057e-04,  1.64819377e-03, -3.52438023e-03, ...,
        -3.72843121e-04,  9.66437798e-03,  4.95006958e-01]])

In [48]:
np.allclose(hessian_inv_np, hessian_inv_torch)

True

In [31]:
grad[0:5]

tensor([ 0.0033, -0.0122,  0.0302,  0.0275,  0.0123], device='cuda:0',
       grad_fn=<SliceBackward>)

In [34]:
grad_train_np[0:5]

array([[ 0.00329743],
       [-0.01223877],
       [ 0.0302425 ],
       [ 0.02748611],
       [ 0.01225669]])

In [41]:
grad.detach().cpu().numpy()

array([ 0.00591992,  0.02018611, -0.00332831,  0.01221107, -0.00989707,
       -0.00665452, -0.00427102,  0.00209346,  0.0188704 ,  0.00428703,
        0.02503378, -0.01275739,  0.00223187, -0.01503066, -0.01725799,
       -0.01338011, -0.01882041,  0.01353167,  0.00534762, -0.01243102,
        0.00300622, -0.00950503, -0.00042133,  0.00139931,  0.00658771,
        0.01399224,  0.00953464,  0.00361797, -0.00010538,  0.00686106,
       -0.0063331 ,  0.01616079, -0.00341326, -0.01052228, -0.00745031,
       -0.01869466, -0.00649654,  0.00804677, -0.02489412,  0.01540469,
       -0.00150601,  0.00556058,  0.01468306, -0.03200147, -0.00252593,
       -0.0136926 ,  0.02515369,  0.02822851,  0.00720127, -0.01135744,
       -0.01292115, -0.006658  ,  0.01473373, -0.01669471, -0.02117098,
        0.0052648 , -0.01185602,  0.01252566, -0.01869623,  0.00397442,
        0.01526202,  0.00440582,  0.01276253,  0.03437793, -0.00895927,
        0.00324804,  0.01153268,  0.0210702 ,  0.00449139, -0.01

In [37]:
grad.shape

torch.Size([165])

In [40]:
np.max(np.absolute(grad.detach().cpu().numpy() - grad_train_np.reshape(-1)))

2.5928020477156144e-08

In [42]:
np.allclose(grad.detach().cpu().numpy(), grad_train_np.astype('float32').reshape(-1), atol=1e-07)

True

In [31]:
hessian

tensor([[ 2.0337e+00, -6.0700e-02, -3.9558e-02,  ..., -1.0019e-02,
         -6.8389e-04,  8.9997e-03],
        [-6.0700e-02,  2.1627e+00,  8.4909e-02,  ...,  2.7004e-02,
          2.4767e-03, -2.4367e-02],
        [-3.9558e-02,  8.4909e-02,  2.0593e+00,  ...,  1.3750e-02,
          1.5693e-03, -1.2251e-02],
        ...,
        [-1.0019e-02,  2.7004e-02,  1.3750e-02,  ...,  2.2125e+00,
          3.2489e-02, -1.5025e-01],
        [-6.8389e-04,  2.4767e-03,  1.5693e-03,  ...,  3.2489e-02,
          2.0155e+00, -1.5087e-02],
        [ 8.9997e-03, -2.4367e-02, -1.2251e-02,  ..., -1.5025e-01,
         -1.5087e-02,  2.1267e+00]], device='cuda:0')

In [32]:
hessian_plus_np

array([[ 2.03370837e+00, -6.07003793e-02, -3.95580381e-02, ...,
        -1.00186933e-02, -6.83893100e-04,  8.99971649e-03],
       [-6.07003793e-02,  2.16272883e+00,  8.49091783e-02, ...,
         2.70038396e-02,  2.47674948e-03, -2.43674032e-02],
       [-3.95580381e-02,  8.49091783e-02,  2.05930830e+00, ...,
         1.37496665e-02,  1.56932708e-03, -1.22508816e-02],
       ...,
       [-1.00186933e-02,  2.70038396e-02,  1.37496665e-02, ...,
         2.21252878e+00,  3.24894302e-02, -1.50252208e-01],
       [-6.83893100e-04,  2.47674948e-03,  1.56932708e-03, ...,
         3.24894302e-02,  2.01554430e+00, -1.50870504e-02],
       [ 8.99971649e-03, -2.43674032e-02, -1.22508816e-02, ...,
        -1.50252208e-01, -1.50870504e-02,  2.12672566e+00]])

In [59]:
np.allclose(hessian.cpu().numpy(), hessian_plus_np)

True

In [62]:
features = model(train_task.x, modulation=modulation)
preds = F.linear(features, weight=adapted_params)
loss = inner_loss_func(preds, train_task.y)
l2_loss = 0.5 * l2_lambda * adapted_params.pow(2).sum()
loss += l2_loss
grad = torch.autograd.grad(loss, adapted_params,
            create_graph=True, allow_unused=False)[0]
grad = grad.flatten()
mixed_partial = torch.zeros((len(grad), features.shape[0] * features.shape[1]), device=grad.device)
for i, y in enumerate(grad):
    mixed_partial[i, :] = torch.autograd.grad(y, features,
            create_graph=False, allow_unused=False, retain_graph=True)[0].flatten()

In [64]:
J

array([[ 3.9498000e-03,  1.6587512e-05, -2.9598630e-05, ...,
         1.3377637e-04,  1.3570306e-04,  1.5941454e-05],
       [-7.7196601e-04,  4.7613578e-03, -1.4194131e-03, ...,
        -2.6781455e-04, -2.7167171e-04, -3.1914107e-05],
       [-6.8175641e-04,  7.0250512e-04,  2.7123529e-03, ...,
        -4.5628636e-04, -4.6285798e-04, -5.4373344e-05],
       ...,
       [ 7.4577914e-04, -3.5770357e-04,  1.6076142e-07, ...,
         4.5776856e-03,  1.1610718e-04,  1.0581916e-04],
       [-1.0331367e-03,  4.9553101e-04, -2.2270470e-07, ...,
        -5.2922819e-04,  4.0567494e-03, -1.6449738e-04],
       [ 3.2615627e-03, -1.5643676e-03,  7.0306800e-07, ...,
         3.9397739e-04,  1.3436373e-04,  4.3596979e-03]], dtype=float32)

In [65]:
 mixed_partial.cpu().numpy()

array([[ 3.9497991e-03,  1.6587504e-05, -2.9598619e-05, ...,
         1.3377628e-04,  1.3570298e-04,  1.5941445e-05],
       [-7.7196554e-04,  4.7613564e-03, -1.4194124e-03, ...,
        -2.6781438e-04, -2.7167154e-04, -3.1914093e-05],
       [-6.8175606e-04,  7.0250488e-04,  2.7123522e-03, ...,
        -4.5628616e-04, -4.6285772e-04, -5.4373311e-05],
       ...,
       [ 7.4577925e-04, -3.5770357e-04,  1.6068952e-07, ...,
         4.5776824e-03,  1.1610711e-04,  1.0581909e-04],
       [-1.0331367e-03,  4.9553096e-04, -2.2263339e-07, ...,
        -5.2922789e-04,  4.0567461e-03, -1.6449727e-04],
       [ 3.2615629e-03, -1.5643674e-03,  7.0287206e-07, ...,
         3.9397710e-04,  1.3436365e-04,  4.3596942e-03]], dtype=float32)

In [63]:
np.allclose(J, mixed_partial.cpu().numpy())

True

In [45]:
p

tensor([[0.1292, 0.1574, 0.2472, 0.1614, 0.3048],
        [0.1900, 0.1465, 0.1348, 0.2250, 0.3036],
        [0.1009, 0.1636, 0.2977, 0.0941, 0.3436],
        [0.0937, 0.0276, 0.0756, 0.1945, 0.6086],
        [0.1112, 0.0810, 0.0660, 0.2441, 0.4977],
        [0.2215, 0.0707, 0.3801, 0.1587, 0.1690],
        [0.1064, 0.1202, 0.5492, 0.1128, 0.1113],
        [0.0899, 0.1706, 0.3484, 0.1766, 0.2144],
        [0.0413, 0.2350, 0.5711, 0.0413, 0.1113],
        [0.1816, 0.1038, 0.3439, 0.1960, 0.1746],
        [0.2108, 0.1655, 0.1266, 0.3371, 0.1600],
        [0.1711, 0.0625, 0.0424, 0.5011, 0.2228],
        [0.2685, 0.0962, 0.1753, 0.2987, 0.1613],
        [0.0669, 0.2088, 0.1623, 0.4196, 0.1424],
        [0.1508, 0.1588, 0.1939, 0.2596, 0.2370],
        [0.4489, 0.2715, 0.0448, 0.1507, 0.0840],
        [0.4626, 0.0360, 0.1035, 0.2422, 0.1557],
        [0.6012, 0.0507, 0.1419, 0.1091, 0.0971],
        [0.1886, 0.1481, 0.1489, 0.2232, 0.2912],
        [0.5540, 0.0600, 0.2184, 0.1101, 0.0575],


In [39]:
grad

tensor([[ 1.0375e-02,  3.2603e-03, -2.7199e-02,  2.7819e-03, -5.3182e-03,
         -1.6668e-02, -4.1722e-03, -5.9818e-03,  3.1078e-02, -1.2360e-02,
          6.0025e-03, -1.4896e-03,  2.0239e-02,  2.7659e-02,  1.2721e-02,
         -1.8344e-02,  1.4714e-02, -1.6974e-02, -2.1356e-03,  1.7683e-02,
          2.8527e-02, -1.0274e-03,  6.9201e-05,  3.7571e-03,  1.7770e-02,
         -2.7123e-02, -1.8309e-02, -1.6115e-02,  9.3267e-03, -2.0881e-02,
         -2.1858e-02, -1.2520e-02, -1.7136e-03],
        [-1.2586e-03, -9.6823e-03,  3.2016e-02, -2.6150e-03,  2.0836e-02,
          4.2606e-03,  6.4458e-04, -6.3008e-03, -2.8042e-04,  3.2261e-02,
         -3.9961e-03,  2.0073e-02, -2.5400e-02, -2.7498e-02,  6.5519e-04,
          1.3939e-02, -1.2225e-04,  5.5207e-04,  2.5551e-02, -1.0602e-02,
         -9.1193e-03,  3.1496e-02, -1.8126e-02,  1.7192e-02, -1.5655e-02,
         -1.0358e-02, -2.9441e-02,  8.5946e-03, -2.6201e-02,  1.4873e-02,
         -1.9204e-02, -2.9080e-03, -6.5468e-04],
        [-8.57

In [32]:
print(loss)

tensor(0.8878, device='cuda:0', grad_fn=<NllLossBackward>)


In [99]:
np.eye(5) - p[0]

array([[ 0.87082767, -0.15739805, -0.24721174, -0.16138791, -0.30482997],
       [-0.12917233,  0.84260195, -0.24721174, -0.16138791, -0.30482997],
       [-0.12917233, -0.15739805,  0.75278826, -0.16138791, -0.30482997],
       [-0.12917233, -0.15739805, -0.24721174,  0.83861209, -0.30482997],
       [-0.12917233, -0.15739805, -0.24721174, -0.16138791,  0.69517003]])

In [107]:
w[:,:-1].shape

(5, 32)

In [109]:
weighted_w = np.matmul(np.diag(p[0]), np.matmul(np.eye(5) - p[0], w[:,:-1]))

In [110]:
weighted_w.shape

(5, 32)

In [115]:
np.outer(X[0], weighted_w[0]).shape

(33, 32)

In [112]:
X = lr_features

In [102]:
w = lr_model.coef_

In [103]:
w.shape

(5, 33)

In [100]:
p[0]

array([0.12917233, 0.15739805, 0.24721174, 0.16138791, 0.30482997])

In [93]:
d = 32
I_d = np.eye(d)
I_0 = np.concatenate((I_d, np.zeros((1, d))), axis=0)

In [94]:
I_d.shape

(32, 32)

In [96]:
I_0.shape

(33, 32)

In [89]:
I = np.eye(p.shape[1])

In [92]:
p - I[y]

array([[ 0.12917233,  0.15739805,  0.24721174,  0.16138791, -0.69517003],
       [ 0.19004614,  0.14653752,  0.13478738,  0.22501634, -0.69638738],
       [ 0.10092306,  0.16361642,  0.29770548,  0.09413159, -0.65637656],
       [ 0.09371822,  0.02756   ,  0.07556733,  0.19453887, -0.39138442],
       [ 0.11122848,  0.08099569,  0.06602907,  0.24408181, -0.50233505],
       [ 0.22151606,  0.07070518, -0.61989829,  0.15868594,  0.1689911 ],
       [ 0.10642068,  0.12024722, -0.45079241,  0.11283058,  0.11129393],
       [ 0.08993535,  0.17059314, -0.65155685,  0.17658611,  0.21444225],
       [ 0.04128719,  0.23497797, -0.42892194,  0.04132418,  0.1113326 ],
       [ 0.18163212,  0.10380414, -0.65608482,  0.19604585,  0.17460272],
       [ 0.21076918,  0.16550224,  0.12660212, -0.66285714,  0.15998359],
       [ 0.17111865,  0.06252387,  0.04244808, -0.49889053,  0.22279993],
       [ 0.26854645,  0.09615933,  0.17531459, -0.70134651,  0.16132614],
       [ 0.06688206,  0.20878349,  0.1

In [36]:
y.cpu().numpy()

array([4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1])

In [83]:
compute_grad(p, y, lr_features)

array([[ 1.03752656e-02],
       [ 3.26025820e-03],
       [-2.71994808e-02],
       [ 2.78191296e-03],
       [-5.31817412e-03],
       [-1.66675723e-02],
       [-4.17220927e-03],
       [-5.98176953e-03],
       [ 3.10775790e-02],
       [-1.23596355e-02],
       [ 6.00250025e-03],
       [-1.48962297e-03],
       [ 2.02389913e-02],
       [ 2.76586834e-02],
       [ 1.27212922e-02],
       [-1.83438292e-02],
       [ 1.47140926e-02],
       [-1.69737805e-02],
       [-2.13557193e-03],
       [ 1.76827033e-02],
       [ 2.85268824e-02],
       [-1.02740732e-03],
       [ 6.92209131e-05],
       [ 3.75709287e-03],
       [ 1.77701430e-02],
       [-2.71230925e-02],
       [-1.83094884e-02],
       [-1.61149143e-02],
       [ 9.32669737e-03],
       [-2.08813414e-02],
       [-2.18583890e-02],
       [-1.25200810e-02],
       [-1.71358201e-03],
       [-1.25856153e-03],
       [-9.68229863e-03],
       [ 3.20163176e-02],
       [-2.61493827e-03],
       [ 2.08360717e-02],
       [ 4.2

In [117]:
I = np.eye(p.shape[1])

In [132]:
I[:, y[0].item(): y[0].item()+1]

array([[0.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [126]:
p[0].reshape(-1, 1)

tensor([[0.1065],
        [0.1636],
        [0.2897],
        [0.2471],
        [0.1931]], dtype=torch.float64)

In [130]:
lr_features[0].shape

torch.Size([33])

In [109]:
a = np.array([[1],
              [0]])
b = np.array([[1],
              [0]])
np.kron(a, b)

array([[1],
       [0],
       [0],
       [0]])

In [67]:
print(compute_hessian(train_task, model, modulation, adapted_params))

TypeError: compute_hessian() takes 3 positional arguments but 4 were given

In [90]:
p = softmax(preds, axis=1)
preds.shape[0]

25

In [92]:
a = np.array([[1,0],
              [0,1]])
b = np.array([[1,1],
              [0,1]])
np.kron(a, b)

array([[1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1]])

In [43]:
def logistic_regression_hessian2_with_respect_to_w(X, y, w):
    '''
    X, y, w numpy arrays
       shape
    X  N, (d+1) last dimension the bias
    y  N integer class identity
    w  C, (d+1) number of classes C
    
    return hessian matrix C*(d+1), C*(d+1)
    '''
    preds = np.matmul(X, w.T)
    p = softmax(preds, axis=1) # the probability matrix N, C

    C = w.shape[0]
    N = X.shape[0]
    d = X.shape[1] - 1
    
    Xbar = np.zeros(shape=(N * (C+1), C * (d+1)))
    diag = []
    
    for i in range(N):
        for j in range(C):
            Xbar[i * C + j, j * (d+1): (j+1) * (d+1)] = X[i]        
        diag.extend(p[i])
    for i in range(N):
        Xbar[N * C + i, :] = np.kron(p[i], X[i])
        diag.append(-1)

    
    result = np.matmul(np.matmul(Xbar.T, np.diag(diag)), Xbar)
    result /= N

    return result

In [47]:
A = logistic_regression_hessian2_with_respect_to_w(X_train, y_train, w)

In [48]:
B = logistic_regression_hessian_with_respect_to_w(X_train, y_train, w)

In [49]:
np.allclose(A, B)

True

In [33]:
X = X_train

In [34]:
y = y_train

In [40]:
preds = np.matmul(X, w.T)
p = softmax(preds, axis=1)

In [42]:
X[0]

array([-0.38918793,  1.7624263 ,  1.1465982 ,  1.7255745 ,  0.70516425,
        2.011344  , -0.90128756, -1.4994539 ,  0.50038975,  2.252832  ,
       -0.13191003, -1.1027367 ,  0.01333237,  0.84084433,  0.27218148,
       -0.5713475 , -2.1324117 ,  0.5417987 , -1.0578914 , -0.7511769 ,
        1.851747  ,  0.80861795, -0.16225591,  0.8148676 ,  2.6477244 ,
       -0.25002277, -1.2813526 ,  0.20728773,  0.4474508 , -3.0171328 ,
       -1.310616  , -0.03914583,  1.        ], dtype=float32)

In [41]:
np.kron(p[0], X[0])

array([-6.37947721e-03,  2.88892779e-02,  1.87947694e-02,  2.82852128e-02,
        1.15588866e-02,  3.29694785e-02, -1.47736939e-02, -2.45786961e-02,
        8.20227154e-03,  3.69278938e-02, -2.16223812e-03, -1.80758014e-02,
        2.18541027e-04,  1.37829231e-02,  4.46153479e-03, -9.36539378e-03,
       -3.49539928e-02,  8.88103712e-03, -1.73407067e-02, -1.23131154e-02,
        3.03534027e-02,  1.32546760e-02, -2.65966076e-03,  1.33571187e-02,
        4.34008762e-02, -4.09831433e-03, -2.10036319e-02,  3.39781190e-03,
        7.33450847e-03, -4.94561307e-02, -2.14833096e-02, -6.41669205e-04,
        1.63917653e-02, -2.80059259e-02,  1.26824021e-01,  8.25091004e-02,
        1.24172181e-01,  5.07435538e-02,  1.44736126e-01, -6.48565665e-02,
       -1.07900560e-01,  3.60080004e-02,  1.62113577e-01, -9.49223340e-03,
       -7.93528259e-02,  9.59395838e-04,  6.05070777e-02,  1.95861533e-02,
       -4.11141105e-02, -1.53448150e-01,  3.89877856e-02, -7.61257634e-02,
       -5.40546179e-02,  